Housing data

https://github.com/aepyornis/nyc-db

https://sql.nycdb.info/


These are some ideas I am working on.

**NOTE: The research took me longer than expected. I am still doing some so I will update the tables and the contents in this notebook during the weekend.**



**First approach**: The buildings where you would not like to live

Violations data:

    HPD Violations dataset
    * Buildings with more HPD Violations since 2013
    

Possible outcomes:

    * Frame by borough?
    * JOIN with [dataset] to know who is the landlord
    * Is there any real state firm supporting any of the Governonr
    
**Second approach**: Why real state firms are so interested in Andrew Como reelection for governor of New York?

"Andrew Cuomo raised $6 million for his 2018 election campaign in the last six months, and at least $1.24 million of that came from New York City area real estate interests, a review of state elections records shows"
https://therealdeal.com/issues_articles/the-nightmare-candidate-for-new-york-real-estate/
https://therealdeal.com/2018/01/17/real-estate-throws-tons-of-cash-at-the-cuomo-campaign-again/

Find which companies are funding Cuomo's campaign and see how many buildings own in NYC. Also, keep record of their violations, etc. to give context.

Ownership data:

    PLUTO
       * OWNERSHIP, OWNER NAME (OwnerName)
       Ref: 
       https://www1.nyc.gov/assets/hpd/downloads/pdf/about/hpd-violation-open-data-2017.pdf
       https://www1.nyc.gov/assets/planning/download/pdf/data-maps/open-data/pluto_datadictionary.pdf?v=17v1_1
       
    HPD REGISTRATIONS ???
       * Owners of buildings with 3+ units are required to register every year with HPD. When buildings change hands, the new owners must update their registration - otherwise they must update it once per year.
       
    ACRIS
       * Detailed records about ownership and transfers of ownership
       
    Check also:
    hpd_business_addresses
    hpd_corporate_owners tables
   
Violations data:

    HPD Violations dataset
    
Donors data:

    Cuomo
    https://www.followthemoney.org/entity-details?eid=6466806
    Reference (Politico): https://www.politico.com/states/new-york/albany/story/2017/07/24/the-18-donors-who-have-given-cuomo-more-than-100k-113571
   
Check if the situation is the same for local politics in districts and neighborhoods.

Datasests contained in nyc-db:

* Department of City Planning's Pluto: versions 16v2 and 17v1

* DOB Job Filings

* DOB Complaints

* HPD Violations

* HPD Registrations

* HPD Complaints

* Department of Finance Rolling Sales

* Tax bills - Rent Stabilization Unit Counts (John Krauss's data)

* ACRIS

# SQL

### Approach 1

First, I have started extracting the violations in the buildings.

Here it is the SQL code written for a first review:

```sql
SELECT
  housenumber,
  streetname,
  apartment,
  bbl,
  longitude,
  latitude,
  novissueddate,
  novdescription,
  violationid,
  violationstatus
FROM hpd_violations
```


To frame the search by zip code once I have a more specific approach, a sample query for pluto db:
    
```sql
SELECT complaintcategory, status, complaintnumber, inspectiondate, housenumber, housestreet
FROM dob_complaints
WHERE bin in (SELECT bin FROM pluto_17v1 WHERE zipcode = '10029' OR zipcode = '10035')
```


Pending: Create a JOIN query to obtain the ownership of the building

# Pandas

Playing with violations in pandas after downloading HPD Violations dataset

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('violations.csv')

/Users/abr/.pyenv/versions/3.6.5/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,ViolationID,BuildingID,RegistrationID,BoroID,Borough,HouseNumber,LowHouseNumber,HighHouseNumber,StreetName,StreetCode,...,NovType,ViolationStatus,Latitude,Longitude,CommunityBoard,CouncilDistrict,CensusTract,BIN,BBL,NTA
0,10000009,265980,301467,3,BROOKLYN,355,355,355,EAST 48 STREET,36930,...,Original,Close,40.653217,-73.932480,17.0,41.0,870.0,3102071.0,3.046740e+09,East Flatbush-Farragut
1,10000011,73852,226626,2,BRONX,1123,1123,1123,EAST TREMONT AVENUE,29620,...,Original,Close,40.839954,-73.876599,6.0,15.0,220.0,2042428.0,2.040040e+09,East Tremont
2,10000012,80102,211704,2,BRONX,751,751,751,GERARD AVENUE,35020,...,Original,Close,40.824692,-73.926605,4.0,8.0,63.0,2002971.0,2.024820e+09,West Concourse
3,10000014,268639,350942,3,BROOKLYN,54,54,54,EAST 52 STREET,37080,...,Original,Close,40.659731,-73.929295,17.0,41.0,878.0,3099672.0,3.046050e+09,Prospect Lefferts Gardens-Wingate
4,10000017,268167,300713,3,BROOKLYN,146,146,146,EAST 52 STREET,37080,...,Original,Close,40.657615,-73.929067,17.0,41.0,878.0,3100214.0,3.046210e+09,Prospect Lefferts Gardens-Wingate


violations.csv is quite heavy! Narrow the query!